#### Code Outline:

This code is for contour extraction from pots with either one handle or no handle.

Note that for those pots that have handles on one side, we need to note this down beforehand in order to 
choose the side that doesn't contain the handle. At present, the algorithm simply chooses the shortest side.

Additionally, if a pot photo is too dark (bg is too dark), then we would want to gather the contours from the original
picture rather than the b/w version this algorithm makes. Thus, we also need to note down whether pots are too dark.

In theory, perhaps a .txt or .csv file should be made with labels such as "dark" or "left" etc for each pot.

In [1]:
import matplotlib.pyplot as plt 
import numpy as np
import pylab as pl
import pandas as pd
from math import sqrt
from skimage.filters import threshold_otsu
from skimage import measure
import os
import re
import itertools as it
import csv
from PIL import Image
from skimage.color import rgb2gray
from skimage.filters import gaussian
from skimage.segmentation import active_contour
from skimage import data, img_as_float
from skimage.segmentation import (morphological_chan_vese,
                                  morphological_geodesic_active_contour,
                                  inverse_gaussian_gradient,
                                  checkerboard_level_set)
pl.ion()

### Contour Functions

In [2]:
def get_outline_contour(img):
    
    # Get all contours:
    thresh = threshold_otsu(img)
    binary = img > thresh
    cont = measure.find_contours(binary, 0.8)
    
    # Find longest contour:
    cont_ln = []
    for n, contour in enumerate(cont):
        cont_ln.append(len(contour))

    longest_c = sorted(cont_ln,reverse=True)[:20]
    long_ind = []
    for i in range(0,len(cont_ln)):
        if cont_ln[i] in longest_c:
            long_ind.append(i)
            
    k = np.argmax(cont_ln)
            
    return cont[k]

In [75]:
def binary_coloured(img):
    x_len = len(img[0])
    y_len = len(img)

    x = []
    y = []

    # Find coordinates of red parts (contour)
    for i in range(0,y_len):
        for j in range(0,x_len):
            if (img[i][j][0] < 155) and (img[i][j][1] < 155) and (img[i][j][2] < 155):
                img[i][j] = [0,0,0]
                y.append(i)
                x.append(j)
            else:
                img[i][j] = [255,255,255]
    return img

In [82]:
def find_centre(x,y):
    yc = min(y)+(max(y)-min(y))/2
    tnth = max(y)-(max(y)-min(y))/10
    coords = np.where((np.array(y)<tnth) & (np.array(y)>yc)) [0]
    xc = min(np.array(x)[coords])+(max(np.array(x)[coords])-min(np.array(x)[coords]))/2
    return xc,yc

In [60]:
def get_contour_side(X,Y,D):
    
    xc,yc = find_centre(X,Y)    
    
    tnth = min(X) + (max(X)-min(X))/10
    lb = xc - tnth
    ub = xc + tnth
    rng = []
    
    rng = np.where((X >= lb) & (X <=ub))[0]
    
    ktop = np.argmax(Y[rng])
    top_pnt = rng[ktop]
    
    kbot = np.argmin(Y[rng])
    bot_pnt = rng[kbot]

    mx_pnt = max(top_pnt,bot_pnt)
    mn_pnt = min(top_pnt,bot_pnt)
        
    xs1 = list(X[mn_pnt:mx_pnt])
    ys1 = list(Y[mn_pnt:mx_pnt])

    xs2 = list(X)[mx_pnt:] + list(X)[:mn_pnt] 
    ys2 = list(Y)[mx_pnt:] + list(Y)[:mn_pnt] 
    
    if D == "R":
        xs = xs2
        ys = ys2
    else:
        if D == "L":
            xs = xs1
            ys = ys1
        else:
            if min(xs1) > min(xs2):
                D1 = "R"
                D2 = "L"
            else:
                D1 = "L"
                D2 = "R"

            if len(ys1) >= len(ys2):
                ys = ys2
                xs = xs2
                D = D2
            else:
                ys = ys1
                xs = xs1
                D = D1


    return xs,ys,xc,yc,D

In [76]:
def edit_pot_ends(x,y,xs,ys,side):
    
    xc,yc = find_centre(x,y)
    
    if side == "unknown":
        if min(np.array(x)[coords]) < xc:
            side = "L"
        else:
            side = "R"
    
    if side == "L":
        coords2 = np.where(np.array(xs)<xc)[0]
    else:
        coords2 = np.where(np.array(xs)>xc)[0]
        
    xnew = []
    ynew = []
    xnew.append(xc)
    ynew.append(ys[0])
    xnew.extend(np.array(xs)[coords2])
    ynew.extend(np.array(ys)[coords2])
    xnew.append(xc)
    ynew.append(ys[-1])
    
    if side == "L":
        xnew = [(-1*i)+2*xc for i in xnew]

    return xnew,ynew

In [65]:
def smooth_side_contour(x,y,direction):
    
    ys = sorted(y)
    ys = np.unique(np.round(ys))
    y = np.round(y)
    x = np.round(x)
        
    xs = []
    for i in range(0,len(ys)):
        inds = np.where(y==ys[i])
        xy = x[inds]
        if direction == 'R':
            xs.append(max(xy))
        else:
            xs.append(min(xy))
            
    return xs,ys

### Test Contours

In [71]:
directory = os.fsencode('C:\\Users\\arian\\OneDrive\\Pictures\\pots\\No_or_one_hnd')

direc_fold = 'C:\\Users\\arian\\OneDrive\\Pictures\\pots\\No_or_one_hnd'

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    nme = str(filename)[:-4]
    try:
        image_grey = data.load(direc_fold+'\\'+str(filename),as_gray=True)
        contour = get_outline_contour(image_grey)
        xcont1 = contour[:,1]
        ycont1 = contour[:,0]

        img = data.load(direc_fold+'\\'+str(filename),as_gray=False)

        x_len = len(img[0])
        y_len = len(img)

        x = []
        y = []

        # Find coordinates of red parts (contour)
        for i in range(0,y_len):
            for j in range(0,x_len):
                if (img[i][j][0] < 155) and (img[i][j][1] < 155) and (img[i][j][2] < 155):
                    img[i][j] = [0,0,0]
                    y.append(i)
                    x.append(j)
                else:
                    img[i][j] = [255,255,255]

        fig, ax = plt.subplots(figsize=(4, 6))
        ax.imshow(img)
        ax.set_axis_off()
        plt.savefig("pot_new.jpg")
        plt.close()

        image_grey2 = data.load('C:\\Users\\arian\\Documents\\GitHub\\pots\\Code\\pot_new.jpg',as_gray=True)

        contour = get_outline_contour(image_grey2)
        xcont2 = contour[:,1]
        ycont2 = contour[:,0]
        
        # Find Centres:
        
        xs1,ys1,xc1,yc1,D1 = get_contour_side(xcont1,ycont1,"Z")
        xs2,ys2,xc2,yc2,D2 = get_contour_side(xcont2,ycont2,"Z")
        x1,y1 = smooth_side_contour(xs1,ys1,D1)
        x2,y2 = smooth_side_contour(xs2,ys2,D2)

        
        if len(xcont1) >= len(xcont2):

            fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(5, 7))
            axes[0].imshow(image_grey,cmap='gray')
            axes[0].plot(xcont1,ycont1,'-r',xc1,yc1,'bs',x1,y1,'--b')
            axes[0].title.set_text('L1 = '+str(len(x1)))
            axes[1].imshow(image_grey2,cmap="gray")
            axes[1].plot(xcont2,ycont2,'-g',xc2,yc2,'bs',x2,y2,'--b')
            axes[1].title.set_text('L2 = '+str(len(x2)))
            fig.tight_layout()
            
        else:
            
            fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(5, 7))
            axes[0].imshow(image_grey,cmap='gray')
            axes[0].plot(xcont1,ycont1,'-g',xc1,yc1,'bs',x1,y1,'--b')
            axes[0].title.set_text('L1 = '+str(len(x1)))
            axes[1].imshow(image_grey2,cmap="gray")
            axes[1].plot(xcont2,ycont2,'-r',xc2,yc2,'bs',x2,y2,'--b')
            axes[1].title.set_text('L2 = '+str(len(x2)))
            fig.tight_layout()
            
    
            
            

        plt.savefig(nme+"_outline_options.png")
        plt.close()
        print("\nExtracted from pot "+nme)
    except:
        print("\nExtraction from pot "+nme+" unsuccessful")

    


Extracted from pot alabastron_1013561

Extracted from pot alabatron_1005104

Extracted from pot aryballos_1006659

Extracted from pot aryballos_1011358

Extracted from pot aryballos_ovoid_1008381

Extracted from pot bowl_1008233

Extracted from pot bowl_1009804

Extracted from pot chalice_9034585

Extracted from pot chous_1001955

Extracted from pot chous_12119

Extracted from pot cup_mastoid_306974

Extracted from pot cup_siana_306522

Extraction from pot dish_9032873 unsuccessful

Extracted from pot dish_9034605

Extracted from pot hydria_8296

Extracted from pot jar_9005664

Extracted from pot jug_9003800

Extracted from pot jug_9005174

Extracted from pot jug_9015969

Extraction from pot jug_9032197 unsuccessful

Extraction from pot krater_calyx_12111 unsuccessful

Extracted from pot kyathos_1009331

Extraction from pot kyathos_352364 unsuccessful

Extraction from pot kyathos_9030432 unsuccessful

Extracted from pot kyathos_9930

Extracted from pot lebes_1003192

Extracted from po

### Extract Contours

In [79]:
directory = os.fsencode('C:\\Users\\arian\\OneDrive\\Pictures\\pots\\No_or_one_hnd')

direc_fold = 'C:\\Users\\arian\\OneDrive\\Pictures\\pots\\No_or_one_hnd'

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    nme = str(filename)[:-4]
    
    print("\nExtracting from pot: "+nme)

    xcont = []
    ycont = []
    
    # 1) Find longest contour from oringal pot image. We assume/hope this is the outline of the pot.
    image_grey = data.load(direc_fold+'\\'+str(filename),as_gray=True)
    contour = get_outline_contour(image_grey)
    xcont1 = contour[:,1]
    ycont1 = contour[:,0]
    
    # 2) To get a better contour, we adjust the photo to make it B/W, and then get the longest contour of this.
    # Adjusting the photo like this does not always work therefore we add a try & except.
    
    try:
        # 2.1) Get B/W version of original image.
        image_col = data.load(direc_fold+'\\'+str(filename),as_gray=False)
        image_bw = binary_coloured(image_col)
        # 2.2) Create a plot and temporarily save. This will be the new B/W of the pot.
        fig, ax = plt.subplots(figsize=(4, 6))
        ax.imshow(image_bw)
        ax.set_axis_off()
        plt.savefig("pot_new.jpg")
        plt.close()
        
        # 2.3) Get longest contour from new pot image.
        image_grey2 = data.load('C:\\Users\\arian\\Documents\\GitHub\\pots\\Code\\pot_new.jpg',as_gray=True)
        contour = get_outline_contour(image_grey2)
        xcont2 = contour[:,1]
        ycont2 = contour[:,0]
        
        # 2.4) Choose the smallest contour between the two outline contours. 
        
        if len(xcont1) >= len(xcont2):
            xcont = xcont2
            ycont = ycont2
            image_grey = image_grey2
        else:
            xcont = xcont1
            ycont = ycont1
    except:
        xcont = xcont1
        ycont = ycont1
        
    # 3) Get one side of contour. 
    # "L" in the third parameter gets the left side of the pot, "R" the right side, and anything else gets
    # the shortest side.
    xs,ys,xc,yc,D = get_contour_side(xcont,ycont,"Z")
    
    # 4) Smooth the contour
    xsmooth,ysmooth = smooth_side_contour(xs,ys,D)
    
    # 5) Edit contour ends so that they start and end at the centre of the pot.
    x,y = edit_pot_ends(xcont,ycont,xsmooth,ysmooth,D)
        
    # 6) Save plot of pot and side contour.
    fig, ax = plt.subplots(figsize=(4, 6))
    ax.imshow(image_grey,cmap='gray')
    ax.set_axis_off()
    ax.plot(x,y,'-r')
    plt.savefig(nme+"_contour.png")
    plt.close()

    


Extracting from pot: alabastron_1013561

Extracting from pot: alabatron_1005104

Extracting from pot: aryballos_1006659

Extracting from pot: aryballos_1011358

Extracting from pot: aryballos_ovoid_1008381

Extracting from pot: bowl_1008233

Extracting from pot: bowl_1009804

Extracting from pot: chalice_9034585

Extracting from pot: chous_1001955

Extracting from pot: chous_12119

Extracting from pot: cup_mastoid_306974

Extracting from pot: cup_siana_306522

Extracting from pot: dish_9032873

Extracting from pot: dish_9034605

Extracting from pot: hydria_8296

Extracting from pot: jar_9005664

Extracting from pot: jug_9003800

Extracting from pot: jug_9005174

Extracting from pot: jug_9015969

Extracting from pot: jug_9032197

Extracting from pot: krater_calyx_12111

Extracting from pot: kyathos_1009331

Extracting from pot: kyathos_352364

Extracting from pot: kyathos_9030432

Extracting from pot: kyathos_9930

Extracting from pot: lebes_1003192

Extracting from pot: lekythos_5657
